In [ ]:
from pathlib import Path
from astropy import units as u
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import aggregate_downsample
from astropy.timeseries import TimeSeries

import re
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def boundify(a, sigma=2.0):
    mean = a.mean()
    dev = np.std(a)
    
    return (mean - (sigma * dev)).value, (mean + (sigma * dev)).value

def check_filter(labels):
    values = []
    
    for label in labels:
        if re.match(r'.*_B[0-9]+_.*', str(label)):
            values.append('Blue')
        elif re.match(r'.*_V[0-9]+_.*', str(label)):
            values.append('Green')
        elif re.match(r'.*_R[0-9]+_.*', str(label)):
            values.append('Red')
        else:
            values.append('Unknown')
    
    return values


In [ ]:
# constants
working_directory = Path('/home/localadmin')
csv_data = working_directory / 'measurements.csv'
swasp_data = working_directory / ''

# TODO: automatically infer this from the FITS data
period = 10.903858 * u.hour

In [ ]:
t = Table.read(csv_data, format='ascii.csv', delimiter='\t', fast_reader=False)
# curate columns
t['JD_UTC'].unit = u.day
t['time'] = Time(t['JD_UTC'], format='jd')
t['filter'] = check_filter(t['Label'])

for row in t:
    check_filter(row)

# remove unwanted data
t.keep_columns(['time','rel_flux_T1', 'rel_flux_err_T1', 'filter'])
t['rel_flux_T1'].unit = 'mag'

In [ ]:
ts = TimeSeries(t)
t

In [ ]:
ts_folded = ts.fold(period=period)
ts_folded.sort('time')

period.to(u.hour)

In [ ]:
y_min, y_max = boundify(ts_folded['rel_flux_T1'])

plt.figure(figsize=(20,20))
plt.plot(ts_folded.time.jd, ts_folded['rel_flux_T1'], 'k.', markersize=5, linestyle=':')
plt.ylim(bottom=y_min, top=y_max)
plt.xlabel('Time (days)')
plt.ylabel('SAP Flux (e-/s)')

In [ ]:
mean, median, stddev = sigma_clipped_stats(ts_folded['rel_flux_T1'])  
ts_folded['norm_flux_T1'] = ts_folded['rel_flux_T1'] / median
# ts_binned = aggregate_downsample(ts_folded, time_bin_size=0.001 * u.day) 

In [ ]:
ts_red = ts_folded[ts_folded['filter'] == 'Red']
ts_green = ts_folded[ts_folded['filter'] == 'Green']
ts_blue = ts_folded[ts_folded['filter'] == 'Blue']


In [ ]:
y_min, y_max = boundify(ts_folded['norm_flux_T1'])

plt.figure(figsize=(20,20))
plt.plot(ts_red.time.jd, ts_red['norm_flux_T1'], 'r.', markersize=5, linestyle=':')
plt.plot(ts_green.time.jd, ts_green['norm_flux_T1'], 'g.', markersize=5, linestyle=':')
plt.plot(ts_blue.time.jd, ts_blue['norm_flux_T1'], 'b.', markersize=5, linestyle=':')

plt.ylim(bottom=y_min, top=y_max)
plt.xlabel('Time (days)')
plt.ylabel('Normalized flux')